# Imports:

In [50]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from datetime import datetime

# Importing libraries fto switch to the new tab
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd
import re

# Browser preferences:

In [51]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder) # you can predefine where you wanna store things in case its needed
    profile.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    profile.set_preference("browser.download.manager.showWhenStarting", False) #I dont wanna see a pop up for each download so i swith it off
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    # profile.install_addon('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse_2025/booking/booking/ublock_origin-1.55.0.xpi')
    # profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True) #dont want the pdf viewer to open

    options = Options()
    options.profile = profile
    return options

'''Modify the start_up function to include a fixed window size'''
def start_up(link, dfolder, geko_path,donwload=True, window_size='1920,1080'):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    width, height = map(int, window_size.split(','))
    browser.set_window_size(width, height)
    # Enter the website address here
    browser.get(link)
    time.sleep(3)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

# Walking through Booking:

In [52]:
# lets open booking:

dfolder='/Users/matiasborrell/Library/Mobile Documents/com~apple~CloudDocs/Documentacion Master Barcelona/BSE/Term 2/Introduction to Text Mining/Homework01/Downloads_HW1'
geko_path='/Users/matiasborrell/Library/Mobile Documents/com~apple~CloudDocs/Documentacion Master Barcelona/BSE/Term 2/Introduction to Text Mining/gecko/geckodriver'
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)


In [53]:
#Reject Cookies
check_and_click(browser, 'onetrust-reject-all-handler', 'id')

In [54]:
# click on the "Where to go button"
browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()

## City Selection:

In [55]:
# Input Barcelona as the target city.
place = 'Barcelona'
search1 = browser.find_element(by='xpath',value='//*[@id=":rh:"]')
search1.send_keys(place)

In [56]:
# Clicking on the enter date button.
browser.find_element(by=By.XPATH,value='//*[@class="ebbedaf8ac ab26a5d2bd e33c97ff6b"]').click()

## Dates Selection:

In [57]:
# Clicking next until we find the right month.

actual_date = datetime.now()
actual_year = str(actual_date.year)

year = '2025'
month = 'marzo'
month_number = '03'
arrival_day = '12'
departure_day = '15'

# We can replace with a specific year or with the os actual year just replacing the variable year by actual_year. (Notice booking just allows you to do reservations up to one year foward.)
target_date = month + ' ' + actual_year

while browser.find_element(By.XPATH, '//h3[contains(@class, "e1eebb6a1e ee7ec6b631")]').text.strip() != target_date:
    browser.find_element(By.XPATH, '//*[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 f4552b6561 dc72a8413c f073249358"]').click()

In [58]:
# Scrolling down 300 Pixels.
browser.execute_script("window.scrollBy(0, 300);")

In [59]:
#Clicking on the desired dates.
path='//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa ef091eb985"]'
dates = browser.find_elements('xpath',path)
for date in dates:
    if date.get_attribute("data-date") == f"{year}-{month_number}-{arrival_day}":
        date.click()
    if date.get_attribute("data-date") == f"{year}-{month_number}-{departure_day}":
            date.click()
            break

In [60]:
# click on the "Search button"
browser.find_element(by=By.XPATH,value='//button[@class="a83ed08757 c21c56c305 a4c1805887 f671049264 a2abacf76b c082d89982 cceeb8986b b9fd3c6b3c"]').click()

In [61]:
# Close offer pop up if it appears.

# Wait a few seconds to see if the pop up appears.
time.sleep(1)
try:
    browser.find_element(by=By.XPATH,value='//div[@class="eb33ef7c47"]//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 f4552b6561"]').click()
except NoSuchElementException:
    pass
time.sleep(5)

# Loading all the hotels available:

I go for a duplicate block here because sometimes the first scroll fails.

In [62]:
i=0
# Scrolling based on the height of the page.
while True:
    try:
        total_height = browser.execute_script("return document.body.scrollHeight")
        scroll_position = (total_height*10) - 200
        browser.execute_script(f"window.scrollTo(0, {scroll_position});")
        
        # Small wait to let the page load.
        time.sleep(2)
        
        # Clicking on the load more button.
        browser.find_element(By.XPATH, '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]').click()
        i += 1
    except NoSuchElementException:
        break

print(f'The load more button has been clicked {i} times')

The load more button has been clicked 0 times


In [ ]:
i=0
# Scrolling based on the height of the page.
while True:
    try:
        total_height = browser.execute_script("return document.body.scrollHeight")
        scroll_position = (total_height*10) - 200
        browser.execute_script(f"window.scrollTo(0, {scroll_position});")
        
        # Small wait to let the page load.
        time.sleep(2)
        
        # Clicking on the load more button.
        browser.find_element(By.XPATH, '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]').click()
        i += 1
    except NoSuchElementException:
        break

print(f'The load more button has been clicked {i} times')

The load more button has been clicked 37 times


# Data extraction for each hotel:

In [65]:
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Creating empty lists to store the data just in case we need it.
full_names = []
full_prices = []
full_score = []
full_descriptions = []

# Creating the DF that is gonna store the data.
df = pd.DataFrame(columns=['Name', 'Price', 'Score', 'Description'])

# Sabing the original handler to be able to jump from one tab to another.
original_window = browser.current_window_handle

hotels_xpath = '//div[@class="f6431b446c a15b38c233"]'

# Finding the full hotel list.
hotels = browser.find_elements(By.XPATH, hotels_xpath)

for index in range(10):
#'''Use this for loop instead to do the full scrapping. It is gonna take a while......... (Around 4 Hours)'''
#for index in range(len(hotels)):
    try:
        # Finding the updated list on each itaration to avoid crashes since we are not working with id.
        hotels = browser.find_elements(By.XPATH, hotels_xpath)

        # Srcolling to the hotel position according to index.
        browser.execute_script("arguments[0].scrollIntoView();", hotels[index])
        time.sleep(2)

        # Clicking on the hotel.
        hotels[index].click()
        time.sleep(3)

        # Switching to the new tab using the handler.
        new_window = [window for window in browser.window_handles if window != original_window][0]
        browser.switch_to.window(new_window)

        # Getting the hotel name.
        try:
            names = str(browser.find_element(By.XPATH, '//div[@data-capla-component-boundary="b-property-web-property-page/PropertyHeaderName"]//h2[@class="d2fee87262 pp-header__title"]').text)
            full_names.append(names)
        except Exception as e:
            names = None
            full_names.append(names)

        # Getting the hotel score.
        try:    
            score = float(browser.find_element(By.XPATH, '//div[@id="js--hp-gallery-scorecard"]').get_attribute("data-review-score"))
            full_score.append(score)
        except Exception as e:
            score = None
            full_score.append(score)

        # Getting the hotel description.
        try:
                browser.execute_script("document.body.style.zoom='70%'")
                browser.execute_script("window.scrollBy(0, 400);")
                description = str(browser.find_element(By.XPATH, '//p[@data-testid="property-description"]').text)
                full_descriptions.append(description)
        except Exception as e:
            description = None
            full_descriptions.append(description)

        # Getting the hotel price.
        try:
            browser.execute_script("window.scrollBy(0, 400);")
            price = browser.find_element(By.XPATH, '//span[@class="prco-valign-middle-helper"]').text
            full_prices.append(price)
        except Exception as e:
            price = None
            full_prices.append(price)

        # Appending the data to the DataFrame.
        df.loc[len(df)] = {'Name': names, 'Price': price, 'Score': score, 'Description': description}
        time.sleep(1)
        
        # Closing the new tab and switching back to the original one.
        browser.close()
        browser.switch_to.window(original_window)

    except Exception as e:
        print(f"Error al procesar el hotel {index}: {e}")
        continue

# Printing the Final DataFrame.
df

,Name,Price,Score,Description
0,Hotel Paxton Barcelona,€ 444,8.8,Hotel Paxton Barcelona se encuentra en Barcelo...
1,Sonder La Casa del Sol,€ 440,8.4,El establecimiento Sonder La Casa del Sol ofre...
2,Hotel Lloret Ramblas,€ 366,8.1,El Hotel Lloret Ramblas se encuentra en las fa...
3,NH Barcelona Diagonal Center,€ 366,8.2,Este hotel de diseño está a 15 minutos de la p...
4,Acta BCN 40,€ 368,8.4,El Acta BCN 40 Hotel se encuentra en el centro...
5,"Holiday Inn Express Barcelona City 22@, an IHG...",€ 483,8.3,Hotel acogedor en Barcelona con wifi y desayun...
6,Hostal la Palmera,€ 340,8.7,"Este hostal, de ambiente tranquilo, está situa..."
7,Acta Antibes,€ 427,8.5,El Acta Antibes está en el distrito del Eixamp...
8,Hotel Condado,€ 383,8.5,El Hotel Condado se encuentra a 200 metros de ...
9,Catalonia Albeniz,€ 433,8.4,El Catalonia Albeniz ofrece habitaciones ampli...
